In [1]:
#import the libraries
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jamesworsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
#import the dataset
df = pd.read_csv('data.csv', dtype={"Id": object, "Title": object, "Location": object, "Area": object, "Classification": object, "SubClassification": object})

In [3]:
#verify the contents
df.head(10)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN
5,37404307,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Customer Service - Call Centre,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN
6,37404355,Casual Café All-rounder in Semaphore,Jora Local,2018-10-07T00:00:00.000Z,Adelaide,NaN,Hospitality & Tourism,Waiting Staff,A cafe/restaurant in Semaphore is seeking a ca...,NaN,0,30,NaN
7,37404350,Casual Café All-rounder in Woolloongabba,Jora Local,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Hospitality & Tourism,Waiting Staff,A cafe in Woolloongabba is seeking a casual Ca...,NaN,0,30,NaN
8,37404301,Telemarketer,Reventon Investments,2018-10-07T00:00:00.000Z,Melbourne,CBD & Inner Suburbs,Call Centre & Customer Service,Sales - Outbound,Market leading investment & financial services...,NaN,0,30,NaN
9,37404286,Casual Dish Hand in Brighton,Jora Local,2018-10-07T00:00:00.000Z,Melbourne,Bayside & South Eastern Suburbs,Hospitality & Tourism,Kitchen & Sandwich Hands,A restaurant in Brighton is seeking a casual D...,NaN,0,30,NaN


In [4]:
#get column names
column_names = df.columns
print(column_names)

#get column data types
df.dtypes

Index(['Id', 'Title', 'Company', 'Date', 'Location', 'Area', 'Classification',
       'SubClassification', 'Requirement', 'FullDescription', 'LowestSalary',
       'HighestSalary', 'JobType'],
      dtype='object')


Id                   object
Title                object
Company              object
Date                 object
Location             object
Area                 object
Classification       object
SubClassification    object
Requirement          object
FullDescription      object
LowestSalary          int64
HighestSalary         int64
JobType              object
dtype: object

In [5]:
#check if columns are unique
for i in column_names:
  print('{} is unique: {}'.format(i, df[i].is_unique))

Id is unique: True
Title is unique: False
Company is unique: False
Date is unique: False
Location is unique: False
Area is unique: False
Classification is unique: False
SubClassification is unique: False
Requirement is unique: False
FullDescription is unique: False
LowestSalary is unique: False
HighestSalary is unique: False
JobType is unique: False


In [6]:
#check index values
df.index.values

array([     0,      1,      2, ..., 318474, 318475, 318476])

In [7]:
#Normalize data

#Change to all lowercase
df['Title'] = df['Title'].str.lower()
df['Company'] = df['Company'].str.lower()
#df['Location'] = df['Location'].str.lower()
df['Area'] = df['Area'].str.lower()
#df['Classification'] = df['Classification'].str.lower()
#df['SubClassification'] = df['SubClassification'].str.lower()
df['Requirement'] = df['Requirement'].str.lower()
df['FullDescription'] = df['FullDescription'].str.lower()
df['JobType'] = df['JobType'].str.lower()

#Remove punctuation
df['Title'] = df['Title'].str.replace(r'[^\w\s]+', ' ')
df['Company'] = df['Company'].str.replace(r'[^\w\s]+', ' ')
#df['Location'] = df['Location'].str.replace(r'[^\w\s]+', ' ')
df['Area'] = df['Area'].str.replace(r'[^\w\s]+', ' ')
#df['Classification'] = df['Classification'].str.replace(r'[^\w\s]+', ' ')
#df['SubClassification'] = df['SubClassification'].str.replace(r'[^\w\s]+', ' ')
df['Requirement'] = df['Requirement'].str.replace(r'[^\w\s]+', ' ')
df['FullDescription'] = df['FullDescription'].replace(r'[^\w\s]+', ' ')
df['JobType'] = df['JobType'].str.replace(r'[^\w\s]+', ' ')

#Remove stop words
#stop = stopwords.words('english')
#df['Title'] = df['Title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['Company'] = df['Company'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['Location'] = df['Location'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['Area'] = df['Area'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['Classification'] = df['Classification'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['SubClassification'] = df['SubClassification'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['Requirement'] = df['Requirement'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['FullDescription'] = df['FullDescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#df['JobType'] = df['JobType'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [8]:
#Check number of missing entires in each column
print("Total entries: ", df.shape[0])
print("Missing entries: ")
df.isnull().sum()

Total entries:  318477
Missing entries: 


Id                        0
Title                     0
Company               12004
Date                      0
Location             121248
Area                 195819
Classification       121248
SubClassification    121248
Requirement               7
FullDescription       16175
LowestSalary              0
HighestSalary             0
JobType               16098
dtype: int64

In [9]:
#Remove entries with missing company
df = df[df['Company'].notna()]

#Replace company with unkown
#df.Company.fillna('Unkown', inplace=True)

#Replace missing Requirements with unkown
df.Requirement.fillna('unkown', inplace=True)

#Replace missing Location with unkown
df.Location.fillna('unkown', inplace=True)

#Replace missing Area with unkown
df.Area.fillna('unkown', inplace=True)

In [10]:
#Replace missing classification with most frequent classification for company
df.loc[df['Classification'].isna(), 'Classification'] = df.groupby(['Company'])['Classification'].transform(lambda x: x.mode()[0] if any(x.mode()) else 'ALL_NAN')

#Replace missing subclassification with most frequent for classification
df.loc[df['SubClassification'].isna(), 'SubClassification'] = df.groupby(['Classification'])['SubClassification'].transform(lambda x: x.mode()[0] if any(x.mode()) else 'ALL_NAN')

#Replace missing jobtype with most frequent by sub classification
#slow
df.loc[df['JobType'].isna(), 'JobType'] = df.groupby(['Title'])['JobType'].transform(lambda x: x.mode()[0] if any(x.mode()) else 'ALL_NAN')

#Replace missing Area with most common for location
#dosent work
#df.loc[df['Area'].isna(), 'Area'] = df.groupby(['Location'])['Area'].transform(lambda x: x.mode()[0] if any(x.mode()) else 'ALL_NAN')






In [11]:
df.replace({'ALL_NAN': np.NaN}, regex=True, inplace=True)
print("Total entries: ", df.shape[0])
print("Missing entries: ")
df.isnull().sum()

Total entries:  306473
Missing entries: 


Id                       0
Title                    0
Company                  0
Date                     0
Location                 0
Area                     0
Classification       14434
SubClassification    14434
Requirement              0
FullDescription      15840
LowestSalary             0
HighestSalary            0
JobType               6224
dtype: int64

In [12]:
#Remove reamining entries with missing entries
#Create list comprehension of the columns to lose
columns_to_drop = [column_names[i] for i in [9]]
#Drop unwanted columns
#Drop FullDescription as unhelfpul and majority of entries are missing
df.drop(columns_to_drop, inplace=True, axis=1)
#Drop any rows still with missing values
df.dropna(inplace=True)

In [13]:
df.head(10)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,LowestSalary,HighestSalary,JobType
0,37404348,casual stock replenisher,aldi stores,2018-10-07T00:00:00.000Z,Sydney,north west hills district,Retail & Consumer Products,Retail Assistants,our casual stock replenishers pride themselves...,0,30,casual vacation
1,37404337,casual stock replenisher,aldi stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,unkown,Retail & Consumer Products,Retail Assistants,our casual stock replenishers pride themselves...,0,30,casual vacation
2,37404356,retail sales superstars and stylists wanted ...,lb creative pty ltd,2018-10-07T00:00:00.000Z,Brisbane,cbd inner suburbs,Retail & Consumer Products,Retail Assistants,brand new flagship store opening sunshine plaza,0,30,casual vacation
6,37404355,casual café all rounder in semaphore,jora local,2018-10-07T00:00:00.000Z,Adelaide,unkown,Hospitality & Tourism,Waiting Staff,a cafe restaurant in semaphore is seeking a ca...,0,30,casual vacation
7,37404350,casual café all rounder in woolloongabba,jora local,2018-10-07T00:00:00.000Z,Brisbane,cbd inner suburbs,Hospitality & Tourism,Waiting Staff,a cafe in woolloongabba is seeking a casual ca...,0,30,casual vacation
8,37404301,telemarketer,reventon investments,2018-10-07T00:00:00.000Z,Melbourne,cbd inner suburbs,Call Centre & Customer Service,Sales - Outbound,market leading investment financial services...,0,30,full time
10,37404333,sales and service banking consultant,hays talent solutions,2018-10-07T00:00:00.000Z,Adelaide,unkown,Banking & Financial Services,Banking - Retail/Branch,we currently have an exciting part time opport...,0,30,part time
16,37404332,sales and service banking consultant,hays talent solutions,2018-10-07T00:00:00.000Z,Adelaide,unkown,Retail & Consumer Products,Retail Assistants,we currently have an exciting part time opport...,0,30,part time
19,37404247,assembler,zoom recruitment training,2018-10-07T00:00:00.000Z,Sydney,parramatta western suburbs,"Manufacturing, Transport & Logistics",Assembly & Process Work,our client located in sydney west has roles ...,0,30,full time
20,37404300,procurement officer,perspective recruitment,2018-10-07T00:00:00.000Z,Melbourne,northern suburbs,"Manufacturing, Transport & Logistics","Purchasing, Procurement & Inventory",the role would be ideal for one that has a nat...,0,30,full time


In [20]:
df.Date = df['Date'[0:9]]
df.Date = df.Date.apply(np.datetime64)

In [22]:
df.dtypes
df.head()

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,LowestSalary,HighestSalary,JobType
0,37404348,casual stock replenisher,aldi stores,2018-10-07,Sydney,north west hills district,Retail & Consumer Products,Retail Assistants,our casual stock replenishers pride themselves...,0,30,casual vacation
1,37404337,casual stock replenisher,aldi stores,2018-10-07,Richmond & Hawkesbury,unkown,Retail & Consumer Products,Retail Assistants,our casual stock replenishers pride themselves...,0,30,casual vacation
2,37404356,retail sales superstars and stylists wanted ...,lb creative pty ltd,2018-10-07,Brisbane,cbd inner suburbs,Retail & Consumer Products,Retail Assistants,brand new flagship store opening sunshine plaza,0,30,casual vacation
6,37404355,casual café all rounder in semaphore,jora local,2018-10-07,Adelaide,unkown,Hospitality & Tourism,Waiting Staff,a cafe restaurant in semaphore is seeking a ca...,0,30,casual vacation
7,37404350,casual café all rounder in woolloongabba,jora local,2018-10-07,Brisbane,cbd inner suburbs,Hospitality & Tourism,Waiting Staff,a cafe in woolloongabba is seeking a casual ca...,0,30,casual vacation


In [23]:
df.to_pickle('cleaned_data') 

NameError: name 'dfICT' is not defined